In [1]:
import os
from dotenv import load_dotenv
import mysql.connector
import requests
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import HTML, display
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from mysql.connector import Error

In [2]:
load_dotenv()

True

In [3]:
# MySQL credentials from environment variables
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

In [15]:
# Function to connect to MySQL database
def connect_to_mysql():
    try:
        connection = mysql.connector.connect(
            host=MYSQL_HOST,  # Use variables loaded from dotenv
            database=MYSQL_DATABASE,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None

# Function to insert data into MySQL
def insert_data_to_mysql(df, connection):
    try:
        if connection.is_connected():
            cursor = connection.cursor()
            insert_query = """
                INSERT INTO weather_forecasts (
                    forecast_time, forecast_temperature, forecast_temp_apparent, forecast_humidity, rain_chance, forecast_winds, weather_code, weather_icon
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """
            # Insert each row from the DataFrame
            for row in df.itertuples(index=False):
                cursor.execute(insert_query, (
                    row.time, 
                    row.temperature, 
                    row.temp_apparent, 
                    row.humidity, 
                    row.rain, 
                    row.wind_speed, 
                    row.weather_code, 
                    row.weather_icon
                ))
            connection.commit()
            print(f"Inserted {cursor.rowcount} rows successfully")
    except Error as e:
        print(f"Error during data insertion: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# Ensure connection is established
mysql_connection = connect_to_mysql()

weather_forecasts = pd.DataFrame({
    'time': [datetime.now()],
    'temperature': [20],
    'temp_apparent': [18],
    'humidity': [50],
    'rain': [0],
    'wind_speed': [5],
    'weather_code': ['clear'],
    'weather_icon': ['01d']
})

if mysql_connection:
    insert_data_to_mysql(weather_forecasts, mysql_connection)
else:
    print("Failed to connect to MySQL database.")

Connected to MySQL database
Error during data insertion: 1366 (HY000): Incorrect integer value: 'clear' for column 'weather_code' at row 1
MySQL connection is closed
